# 預處理流程

清emoji、@@username、url -> 存成clean_text

In [5]:
import pandas as pd
import re

In [ ]:
# df = pd.read_csv('comments/face2face_comments_merged.csv', encoding='utf-8')
# print(f"一共有 {len(df)} 筆留言")

一共有 105186 筆留言


In [ ]:
# 取不重複留言者姓名

def get_name_list(data):
    data['author_name'] = data['author_name'].apply(lambda x: str(x).strip())

    name_set = set()
    for name in data['author_name']:
        clean_name = name.strip()
        if clean_name:
            name_set.add(clean_name)

    return list(name_set)


# name_list = get_name_list(df)
# print(">>> name_list:", name_list)  
# print(f"一共有 {len(name_list)} 個不重複留言者")

>>> name_list: ['DavidLee-cf5lw', 'kof78225', '高竿栓民', '洪滿得', 'yateha5122', 'k4-DK', 'ppone8692', 'kenlee3878', '香吉士-q1n', 'kkkuo82167', '買阿哪次不買-n7n', 'allen193', 'lpei8667', '彭春琴', 'priscilla0725', '牛哥哥-m4l', 'YEeEe-g8f', '仁川蝦米郎', '詹順凱-d6n', 'tautjimmylunch7873', 'dehawlee6852', '3n-Just', 'くだる海', 'I_LOVE_MAFUMAFU', 'leebin781202', '張詠翔-v6x', '王明華-g2d', 'lleezi6867', 'shing4352', 'unhero70078', '姜唯一的選哲', 'jingweichen3473', 'sumayyu2851', 'tzengangus660', 'zzzzz7890', 'colomv', 'jonassoviecki6441', 'yehsr3182', 'tyserl89-YT', 'coachashow', 'Chou-d8u', 'gan_rin_nyare', 'etoyocc8556', '司馬信-h8j', 'terrylai5621', 'gastonchen8720', '孫一方', 'susuya1966', '小豆-t2n', '王樹熊', 'chualex7731', 'addasa5702', 'chngray6033', 'ss910404', '蕭正彥-w4k', 'kenkenyulin', '郭依庭-p9e', 'rosehsieh4444', 'a0652qj', '李不克强', 'a129564959', '赵明-z7d', 'uy8057', 'ytc7596', '楊正旭-f8w', '綺綺-1110', 'JayLin-nw5bo', 'TingTeresa', 'goodpeople819', 'chiachia2993', '烟一缕', 'teresayeh3049', 'fayemariah', '林雋庭-n8b', 'LINK3810', '仕任楊', '

In [42]:
def cleaning_text(comment_data, name_list):
    # 在DataFrame中創建一個臨時列來存儲處理過程中的文本
    comment_data['cleaned_text'] = comment_data['comment_text'].copy()
    
    # 1. 從原始留言中移除@@用戶名格式
    if name_list:
        for name in name_list:
            # 為每個名字創建@@name的模式（完整匹配）
            at_pattern = f'@@{re.escape(name)}'
            # 從留言文本中移除@@name
            comment_data['cleaned_text'] = comment_data['cleaned_text'].apply(
                lambda x: re.sub(rf'{at_pattern}', '', str(x))
            )
    
    # 2. 移除HTML標籤 (<a> 標籤和 <br> 等)
    comment_data['cleaned_text'] = comment_data['cleaned_text'].apply(
        lambda x: re.sub(r'<a href=.*?>.*?</a>|<br>|</?b>', ' ', str(x))
    )
    
    # 3. 移除非文字字符和特殊符號
    comment_data['cleaned_text'] = comment_data['cleaned_text'].apply(
        lambda x: re.sub(r'[^\w\s,]', ' ', str(x))
    )
    
    # 4. 只保留中文、英文、數字
    comment_data['cleaned_text'] = comment_data['cleaned_text'].apply(
        lambda x: re.sub(r'[^a-zA-Z0-9\u4e00-\u9fa5]', ' ', str(x))
    )
    
    # 5. 清理多餘的空格
    comment_data['cleaned_text'] = comment_data['cleaned_text'].apply(
        lambda x: re.sub(r'\s+', ' ', x).strip()
    )

    # 6. 去除空白留言
    df_cleaned = comment_data[comment_data['cleaned_text'].str.strip() != '']

    # 7. 統計清除數量
    original = comment_data.groupby('video_title').size()
    after_drop = df_cleaned.groupby('video_title').size().reindex(original.index, fill_value=0)
    drop_count = (original - after_drop).reset_index()
    drop_count.columns = ['video_title', 'count']

    return df_cleaned, drop_count

In [44]:
# 賀瓏分集檢查 'cleaned_text' 的姓名有沒有除乾淨

for i in range(0, 36):
    file_path = f"comments/for_bert/video_{i}_ckip.csv"
    temp_df = pd.read_csv(file_path, encoding='utf-8')
    print(f">>> Processing video_{i} with {len(temp_df)} comments")

    # 取得不重複留言者姓名
    name_list = get_name_list(temp_df)

    # chcek cleaned_text
    cleaned_df, drop_df = cleaning_text(temp_df, name_list)
    cleaned_df.to_csv(f'comments/for_bert/video_{i}_ckip_cleaned.csv', index=False)
    drop_df.to_csv(f'comments/video_{i}_drop_count.csv', index=False)

>>> Processing video_0 with 1948 comments
>>> Processing video_1 with 13270 comments
>>> Processing video_2 with 1513 comments
>>> Processing video_3 with 1099 comments
>>> Processing video_4 with 989 comments
>>> Processing video_5 with 1401 comments
>>> Processing video_6 with 1161 comments
>>> Processing video_7 with 2135 comments
>>> Processing video_8 with 1943 comments
>>> Processing video_9 with 1244 comments
>>> Processing video_10 with 822 comments
>>> Processing video_11 with 1338 comments
>>> Processing video_12 with 881 comments
>>> Processing video_13 with 6784 comments
>>> Processing video_14 with 10795 comments
>>> Processing video_15 with 2435 comments
>>> Processing video_16 with 585 comments
>>> Processing video_17 with 2437 comments
>>> Processing video_18 with 3954 comments
>>> Processing video_19 with 1783 comments
>>> Processing video_20 with 1442 comments
>>> Processing video_21 with 5104 comments
>>> Processing video_22 with 4708 comments
>>> Processing video_23

# 計算互動

- 技術性互動：top_comment/reply/single
- 實質討論：reply/@@username

In [ ]:
# df = pd.read_csv('comments/face2face_cleaned.csv')

In [35]:

def tag_interact(df, name_list):
    """
    top_interaction: 留言串的第一則
    single: 沒有任何互動的單一留言
    reply: comment_type == "reply" or @@usename in comment_text
    """
    video_id_map = {title: idx + 1 for idx, title in enumerate(df['video_title'].unique())}
    df['video_id'] = df['video_title'].map(video_id_map)

    # 建立結果列表
    discussion_results = []
    
    # 針對每部影片獨立處理
    for vid in df['video_id'].unique():
        mask = df['video_id'] == vid  # mask是用來抓出特定影片資料
        print(f"=== 處理影片 {vid} ===")
        sub_df = df[mask].reset_index()

        for i in range(len(sub_df)):
            current_row = sub_df.iloc[i].copy()
            current_type = current_row['comment_type']
            next_type = sub_df.iloc[i + 1]['comment_type'] if i + 1 < len(sub_df) else None

            if current_type == 'top_comment' and next_type == 'reply':
                current_row['interaction_type'] = "top_interaction"
            
            elif current_type == 'top_comment' and next_type == 'top_comment':
                for name in name_list:
                    if f"@@{str(name)}" in current_row['comment_text']:
                        current_row['interaction_type'] = "reply"
                    else:
                        current_row['interaction_type'] = "single"
            
            elif current_type == 'reply':
                current_row['interaction_type'] = "reply"

            discussion_results.append(current_row)
    
    # 將結果轉換為 DataFrame
    result_df = pd.DataFrame(discussion_results)
    
    # 儲存結果到 CSV
    result_df.to_csv('comments/face2face_discussion_group.csv', index=False)
    
    
    return result_df

# interaction_tag = tag_interact(df, names)

In [36]:
# count interaction type
def count_dc_group(df: pd.DataFrame) -> pd.DataFrame:
    result = []

    for vid in df['video_id'].unique():
        sub_df = df[df['video_id'] == vid]
        video_title = sub_df['video_title'].iloc[0]

        # 實質互動留言數：有參與討論串的留言
        interactive_comments = sub_df[sub_df['interaction_type'] != 'single'].shape[0]

        # 總留言數
        total_comments = sub_df.shape[0]

        # 實質互動比例
        interaction_ratio = interactive_comments / total_comments

        result.append({
            'video_id': vid,
            'video_title': video_title,
            'total_comments': total_comments,
            'interactive_comments': interactive_comments,
            'interaction_ratio': round(interaction_ratio, 
                                       4)
        })

    result_df = pd.DataFrame(result)
    result_df.to_csv('comments/face2face_discussion_counts.csv', index=False)

    return result_df

# interaction_counts = count_dc_group(interaction_tag)

# Spam tag

### 過濾掉同一個留言者洗留言 -> 標注為spam
- 以cleaned_text為主

In [45]:
def count_spammer_comments(df):
    user_counts = df['author_name'].value_counts()
    name_count_list = []
    for name, count in user_counts.items():
        if count > 3:
            name_count_list.append({name: count})
    print(">>> 留言數量大於 3 的使用者：", name_count_list)

    # if same user comment the SAME commnet more than 3 times, spam_tag == 'spam', else 'non-spam'ArithmeticError
    df['spam_tag'] = 'non-spam'

    for idx, row in df.iterrows():
        comment_text = row['cleaned_text']
        author_name = row['author_name']

        if author_name in user_counts.index and user_counts[author_name] > 3:
            print(f">>> 正在檢查 {author_name} 的留言")

            if author_name in [list(item.keys())[0] for item in name_count_list]:
                duplicate_comments = df[df['author_name'] == author_name]['cleaned_text'].value_counts()
                
                if duplicate_comments[comment_text] > 3:
                    df.at[idx, 'spam_tag'] = 'spam'
                    print(f">>> username: {author_name} 重複留言超過3次，標記為spam")

    print(">>> spam_tag 統計：")
    spam_counts = df['spam_tag'].value_counts()
    print(spam_counts)
    
    return df 

        


In [46]:
# test
file_path = 'comments/for_bert'

for i in range(0, 36):
    file_name = f'{file_path}/video_{i}_ckip_cleaned.csv'
    df = pd.read_csv(file_name, encoding='utf-8')
    print(f"一共有 {len(df)} 筆留言")
    tag_df = count_spammer_comments(df)
    print(type(tag_df))

    # save to csv
    tag_df.to_csv(f'{file_path}/video_{i}_ckip_spam_tag.csv', index=False)

一共有 1948 筆留言
>>> 留言數量大於 3 的使用者： [{'1taichang': 26}, {'翡翠-c9q': 23}, {'哈哈-z1p': 22}, {'image1191': 19}, {'patrickyang5760': 15}, {'gravitywide': 13}, {'alstonc.8964': 12}, {'user-fukakai': 11}, {'wallowes': 9}, {'rayrayrayray7630': 9}, {'UncleJigen': 9}, {'陳志慈-b6g': 8}, {'allen747': 7}, {'kycia12': 6}, {'kevinfan8020': 6}, {'Chou-d8u': 6}, {'蕭秉科-c1r': 5}, {'超營養雞排-o3e': 5}, {'alicelin661': 5}, {'烟一缕': 4}, {'jtang4378': 4}, {'楊繼芬': 4}, {'kokkeonglai2581': 4}, {'香吉士-q1n': 4}, {'推翻共產黨共產黨才是': 4}, {'0鍇': 4}, {'adsmk22': 4}, {'joyoungjin-yc8sb': 4}, {'bobobo0413': 4}]
>>> 正在檢查 allen747 的留言
>>> 正在檢查 蕭秉科-c1r 的留言
>>> username: 蕭秉科-c1r 重複留言超過3次，標記為spam
>>> 正在檢查 蕭秉科-c1r 的留言
>>> username: 蕭秉科-c1r 重複留言超過3次，標記為spam
>>> 正在檢查 蕭秉科-c1r 的留言
>>> username: 蕭秉科-c1r 重複留言超過3次，標記為spam
>>> 正在檢查 蕭秉科-c1r 的留言
>>> username: 蕭秉科-c1r 重複留言超過3次，標記為spam
>>> 正在檢查 蕭秉科-c1r 的留言
>>> username: 蕭秉科-c1r 重複留言超過3次，標記為spam
>>> 正在檢查 哈哈-z1p 的留言
>>> 正在檢查 哈哈-z1p 的留言
>>> 正在檢查 哈哈-z1p 的留言
>>> 正在檢查 哈哈-z1p 的留言
>>> 正在檢查 哈哈-z1p 的留言
>>> 正在檢查 哈哈

### 人工標注spam/non-spam
spam 的標準：

- 留言過短 （少於5個字）
- 表情符號＞文字
- url > 文字
- 無意義的內容 e.g. 哈哈哈哈哈哈哈、Albee真的好強...、你講支語喔、贺龙好样的

p.s. cleaned_text的表情符號和url已經過濾掉，所以如果文字<5就是spam

In [3]:
"""
人工標注的資料是從每一部影片過濾spammer之後，隨機抽取60則留言組成
"""

import pandas as pd

comments_for_tag = []

for i in range(0, 36):
    file_path = f"comments/for_bert/video_{i}_ckip_spam_tag.csv"
    print(f"Processing video {i} from {file_path}")
    
    temp_df = pd.read_csv(file_path)
    # 計算10%樣本數量
    sample_size = max(int(len(temp_df) * 0.1), 1)  # 至少抽1則留言
    
    # 隨機抽樣10%的數據
    comments_for_tag.append(temp_df.sample(n=sample_size, random_state=42))
    print(f"  抽取了 {sample_size} 則留言 (10%)")


# 確保有收集到資料
if comments_for_tag:
    tag_comments_all = pd.concat(comments_for_tag, ignore_index=True)
    tag_comments_all = tag_comments_all.reset_index(drop=True)
    
    # 建立輸出目錄
    tag_comments_all.to_csv("comments/for_bert/comments_spam_tag.csv", index=False)
    print(f"saved to comments/for_bert/comments_for_tag.csv with {len(tag_comments_all)} comments")
else:
    print("沒有找到任何資料可供處理")

Processing video 0 from comments/for_bert/video_0_ckip_spam_tag.csv
  抽取了 194 則留言 (10%)
Processing video 1 from comments/for_bert/video_1_ckip_spam_tag.csv
  抽取了 1327 則留言 (10%)
Processing video 2 from comments/for_bert/video_2_ckip_spam_tag.csv
  抽取了 151 則留言 (10%)
Processing video 3 from comments/for_bert/video_3_ckip_spam_tag.csv
  抽取了 109 則留言 (10%)
Processing video 4 from comments/for_bert/video_4_ckip_spam_tag.csv
  抽取了 98 則留言 (10%)
Processing video 5 from comments/for_bert/video_5_ckip_spam_tag.csv
  抽取了 140 則留言 (10%)
Processing video 6 from comments/for_bert/video_6_ckip_spam_tag.csv
  抽取了 116 則留言 (10%)
Processing video 7 from comments/for_bert/video_7_ckip_spam_tag.csv
  抽取了 213 則留言 (10%)
Processing video 8 from comments/for_bert/video_8_ckip_spam_tag.csv
  抽取了 194 則留言 (10%)
Processing video 9 from comments/for_bert/video_9_ckip_spam_tag.csv
  抽取了 124 則留言 (10%)
Processing video 10 from comments/for_bert/video_10_ckip_spam_tag.csv
  抽取了 82 則留言 (10%)
Processing video 11 from commen